In [1]:
import tweepy
import cnfg
import time
import pickle

In [3]:
# import access keys
config = cnfg.load("/Users/kaitlynmurphy/Documents/Post-It Project/.twitter_config")

# feed access keys to OAuth handler
auth = tweepy.OAuthHandler(config["consumer_key"],
                           config["consumer_secret"])
auth.set_access_token(config["access_token"],
                      config["access_token_secret"])

# connect to twitter API
api = tweepy.API(auth)

In [4]:
def get_tweets(search_term, n_tweets, sleep_time):
    """This function calls the Twitter API using tweepy
    
    Args:
        search_term: any string to search for on Twitter
        n_tweets: the number of tweets to collect
        sleep_time: the amount of time to wait before re-rerunning (this is to avoid hitting the Twitter rate limit)
    
    Returns:
        A list of tweet status objects 
    """
    tweets = []
    tweets_status = api.search(q=search_term, count=100, result_type='recent')

    for tweet in tweets_status:
        tweets.append(tweet)

    tweets_max = tweets_status.since_id

    while len(tweets) < n_tweets:
        tweets_status = api.search(q=search_term, result_type='recent', since_id=tweets_max, count=100)
        tweets_max = tweets_status.since_id
        for tweet in tweets_status:
            tweets += [tweet]
        print 'appending', len(tweets_status), time.time()
        time.sleep(sleep_time)

    print 'done', len(tweets)
    return tweets

In [5]:
scotus = get_tweets('#SCOTUS', 300, 30)

appending 3 1485908826.87
appending 100 1485908857.49
appending 100 1485908888.22
done 303


In [6]:
tweet_ids = []

for tweet in scotus:
    tweet_ids += [tweet.id]
    
len(tweet_ids), len(set(tweet_ids))

(303, 303)

In [7]:
with open('scotus_tweets_01_31_test.pkl', 'w') as picklefile:
    pickle.dump(scotus, picklefile)